In [1]:
from pprint import pprint
from uuid import uuid4
from pyrsistent import freeze, thaw
from dvent.event import Event

In [2]:
# To create a generic event just use Event.generate
Event.generate('SomethingHappened')

Event(data=pmap({}), version=0, id='20680407-2777-422a-8fd0-e4c06be58f2f', type='SomethingHappened', timestamp=datetime.datetime(2018, 5, 20, 21, 18, 6, 912024), stream_id='')

In [3]:
# Events are PRecords, which are ultimately PMaps
# So they can be easily converted into a python dict
dict(Event.generate('SomethingHappened'))

{'data': pmap({}),
 'version': 0,
 'id': '17b734fe-a6e4-401b-b3d8-6a2e12158d10',
 'type': 'SomethingHappened',
 'timestamp': datetime.datetime(2018, 5, 20, 21, 18, 6, 938206),
 'stream_id': ''}

In [4]:
# For nested pyrsistent data-structures (like Event)
# you should use thaw; note the `data` element this time
thaw(Event.generate('SomethingHappened'))

{'data': {},
 'version': 0,
 'id': 'c1b056ae-5528-4d6e-abd1-6cccd5559846',
 'type': 'SomethingHappened',
 'timestamp': datetime.datetime(2018, 5, 20, 21, 18, 6, 954970),
 'stream_id': ''}

In [5]:
# Events are immutable, you can't change them
event = Event.generate('SomethingHappened', data=freeze({'foo': 'bar'}))
event.version = 1

AttributeError: 'Event' object has no attribute 'version'

In [6]:
# You must create a new instance from the existing
versioned_event = event.set('version', 1)
pprint(event.version)
pprint(versioned_event.version)

0
1


In [7]:
# With this pattern you can easily chain and compose
new_event = event\
  .set('version', 11)\
  .set('data', event.data.set('foo', 'baz'))\
  .set('stream_id', str(uuid4()))

pprint(thaw(new_event))

# And again the original is safely unchanged
pprint(thaw(event))

{'data': {'foo': 'baz'},
 'id': 'b08465f5-73db-46f0-948b-58e9a27ff5c7',
 'stream_id': 'b8817830-3d8c-406d-bd96-f993d113c77e',
 'timestamp': datetime.datetime(2018, 5, 20, 21, 18, 6, 971760),
 'type': 'SomethingHappened',
 'version': 11}
{'data': {'foo': 'bar'},
 'id': 'b08465f5-73db-46f0-948b-58e9a27ff5c7',
 'stream_id': '',
 'timestamp': datetime.datetime(2018, 5, 20, 21, 18, 6, 971760),
 'type': 'SomethingHappened',
 'version': 0}
